In [74]:
import cv2
import pickle
import math
from statistics import mean,median,mode 

%load_ext Cython

import numpy as np
import argparse
import time
import cv2
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imutils

import numpy as np

from skimage.measure import compare_ssim
from PIL import Image
import time

import scipy
import random
import math

# Image manipulation.

import imagehash

from numpy import load

from PIL import Image

from numpy import asarray
from numpy import save

import pandas as pd
import constants
import helper



The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [75]:
from scipy.spatial import distance
def find_distance(point1,point2):
        d = distance.euclidean(point1, point2)
        return d

In [76]:
def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou*100

In [16]:
def get_closest_bounding_box_iou(gt_image_bboxes_list,new_frame_bboxes_list):
    
    bboxes_1st_image=[]
    bboxes_2nd_image=[]
    
    #converting gt_image_bboxes_into_format (x,w,x+w,y+h,class,center_x,center_y)    
    for i in gt_image_bboxes_list:
        bbox_detection = [i[0],i[1],i[0]+i[2],i[1]+i[3],i[4],((i[0]+i[2])/2),((i[1]+i[3])/2)]
        bboxes_1st_image.append(bbox_detection)
        
    #converting new_image_bboxes_into_format (x,w,x+w,y+h,center_x,center_y,class,)
    for i in new_frame_bboxes_list:
        bbox_detection = [i[0],i[1],i[0]+i[2],i[1]+i[3],i[4],((i[0]+i[2])/2),((i[1]+i[3])/2)]
        bboxes_2nd_image.append(bbox_detection)
        
    if len(bboxes_1st_image)>len(bboxes_2nd_image):
        bboxes_1st_image,bboxes_2nd_image=bboxes_2nd_image,bboxes_1st_image
    
    ious=[]
    for i in bboxes_2nd_image:
        center_of_object_bbox_x = i[5]
        center_of_object_bbox_y = i[6]
        class_name              = i[4]
        #print('finding iou for: ',class_name)
        #find the closest bbox of the same class whose center is threhold_pixels apart
        temp_ious=[]
        for j in bboxes_1st_image:
            center_of_object_x_gt = j[5]
            center_of_object_y_gt = j[6]
            class_name_gt         = j[4]
            if class_name_gt == class_name:
                btw_distance=find_distance((center_of_object_bbox_x,center_of_object_bbox_y),(center_of_object_x_gt,center_of_object_y_gt))
                if btw_distance<10:
                    iou_btw = bb_intersection_over_union(j[:4],i[:4])
                    #print(iou_btw)
                    temp_ious.append(iou_btw)
                else:
                    temp_ious.append(0)
            else:
                temp_ious.append(0)
        #print(temp_ious)
        max_iou_for_detection = max(temp_ious)
        #print(max_iou_for_detection)
        ious.append(max_iou_for_detection)
        
    #print(mean(ious))                
    return mean(ious)

In [77]:

def get_flow_vector_count(image_1,image_2):
    frame1 = cv2.imread(image_1)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

    frame2 = cv2.imread(image_2)
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    count = 0
    

    
    #mask = np.zeros_like(frame1)
    for i in range(1, 36):
        for j in range(1,64):
            X1 = (i)*10
            Y1 = (j)*10
            
            
            X2 = int(X1 + flow[X1,Y1,1])
            Y2 = int(Y1 + flow[X1,Y1,0])
            
   
         
            
            if abs(X2-X1)>1 or abs(Y2-Y1)>1:
                count+=1
                #mask = cv2.line(mask, (Y1,X1),(Y2,X2), [0, 255, 255], 1)
                
    return count
               
      
#    img = cv2.add(frame1,mask)
#     cv2.imshow('test',flow)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     return []
                
#     #return count

In [87]:
start =time.time()
count=get_flow_vector_count('frames/0.jpg','frames/9.jpg')
print(time.time()-start)
count


0.24844050407409668


45

In [8]:
with open('yolo_data_640x360.p', 'rb') as handle:
    yolo_data = pickle.load(handle)

In [9]:
detections_1 = yolo_data[40]
detections_1

[[73, 143, 28, 89, 'person', 0.9963099360466003, 0],
 [268, 197, 44, 145, 'person', 0.9624533653259277, 0],
 [291, 108, 26, 81, 'person', 0.9393421411514282, 0],
 [154, 195, 51, 63, 'pottedplant', 0.8549535870552063, 58],
 [270, 221, 41, 94, 'handbag', 0.6627448797225952, 26],
 [419, 274, 64, 63, 'backpack', 0.540287971496582, 24]]

In [10]:
detections_2 = yolo_data[0]

In [11]:
get_closest_bounding_box_iou(detections_2,detections_1)

59.022384245196

In [12]:
df_640x360 = pd.DataFrame(columns=['image'])
images=[i for i in range(0,1083)]
df_640x360['image'] = images

In [13]:
#dp = pickle.load( open( "flow_dynamic_array.data", "rb" ) )
dp=[[-1 for i in range(0,1083)] for i in range(0,1083)]

In [26]:
threshold_list=[25,50,75,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]


for thresh_value in threshold_list[::-1]:
    print('[INFO] starting treshold value of {}'.format(thresh_value))
    
    reference_image = 0
    
    reference_image_list=[]
    
    iou_detection_flag_list=[]
    
    iou_list_between_reference_n_image=[]
    
    number_of_function_calls = 0
    
    flow_count_list=[]


    for i in range(0,1083):
        detections_1 = yolo_data[reference_image]
        image_1_path = '640x360/{}.jpg'.format(reference_image)
       
        
        detections_2 = yolo_data[i]
        image_2_path = '640x360/{}.jpg'.format(i)
        
        reference_image_list.append(reference_image)
        
        if dp[reference_image][i]!=-1:
            flow_count=dp[reference_image][i]

        else:
            flow_count=get_flow_vector_count(image_1_path,image_2_path)
            dp[reference_image][i]=flow_count
            
        flow_count_list.append(flow_count)
        
        
        
        iou=get_closest_bounding_box_iou(detections_1,detections_2)
        iou_list_between_reference_n_image.append(iou)
        
        if iou < thresh_value:
            print('{}.jpg | {}.jpg | IOU: {} |thresh: {} | flow_count: {}| func_call: True '.format(i,reference_image,iou,thresh_value,flow_count))        
            reference_image = i
            iou_detection_flag_list.append(1)
            
        else:
            print('{}.jpg | {}.jpg | IOU: {} |thresh: {} | flow_count: {}| func_call: False '.format(i,reference_image,iou,thresh_value,flow_count))        
            iou_detection_flag_list.append(0)
        

    df_640x360['reference_image_{}'.format(thresh_value)] = reference_image_list
    df_640x360['{}_IOU'.format(thresh_value)]             = iou_list_between_reference_n_image
    df_640x360['{}_IOU_Flag'.format(thresh_value)]        = iou_detection_flag_list
    df_640x360['{}_flow_count'.format(thresh_value)]      = flow_count_list
    



[INFO] starting treshold value of 100
0.jpg | 0.jpg | IOU: 100.0 |thresh: 100 | flow_count: 0| func_call: False 
1.jpg | 0.jpg | IOU: 94.72719359200363 |thresh: 100 | flow_count: 0| func_call: True 
2.jpg | 1.jpg | IOU: 78.92552612481857 |thresh: 100 | flow_count: 0| func_call: True 
3.jpg | 2.jpg | IOU: 98.83797216934009 |thresh: 100 | flow_count: 0| func_call: True 
4.jpg | 3.jpg | IOU: 75.62960846869765 |thresh: 100 | flow_count: 0| func_call: True 
5.jpg | 4.jpg | IOU: 99.53125 |thresh: 100 | flow_count: 0| func_call: True 
6.jpg | 5.jpg | IOU: 79.48768231400906 |thresh: 100 | flow_count: 0| func_call: True 
7.jpg | 6.jpg | IOU: 96.98921822954837 |thresh: 100 | flow_count: 0| func_call: True 
8.jpg | 7.jpg | IOU: 78.33467741935483 |thresh: 100 | flow_count: 0| func_call: True 
9.jpg | 8.jpg | IOU: 79.24764122043867 |thresh: 100 | flow_count: 0| func_call: True 
10.jpg | 9.jpg | IOU: 92.9912520163792 |thresh: 100 | flow_count: 0| func_call: True 
11.jpg | 10.jpg | IOU: 95.9232067197

563.jpg | 562.jpg | IOU: 95.76933770742568 |thresh: 100 | flow_count: 25| func_call: True 
564.jpg | 563.jpg | IOU: 93.95970577505732 |thresh: 100 | flow_count: 35| func_call: True 
565.jpg | 564.jpg | IOU: 86.4475277680375 |thresh: 100 | flow_count: 31| func_call: True 
566.jpg | 565.jpg | IOU: 95.68005104086475 |thresh: 100 | flow_count: 2| func_call: True 
567.jpg | 566.jpg | IOU: 81.25916096351058 |thresh: 100 | flow_count: 53| func_call: True 
568.jpg | 567.jpg | IOU: 95.87412017125996 |thresh: 100 | flow_count: 11| func_call: True 
569.jpg | 568.jpg | IOU: 90.81505292758048 |thresh: 100 | flow_count: 40| func_call: True 
570.jpg | 569.jpg | IOU: 85.98887143873881 |thresh: 100 | flow_count: 13| func_call: True 
571.jpg | 570.jpg | IOU: 64.6253918848922 |thresh: 100 | flow_count: 10| func_call: True 
572.jpg | 571.jpg | IOU: 95.49390218559404 |thresh: 100 | flow_count: 9| func_call: True 
573.jpg | 572.jpg | IOU: 87.5082441626947 |thresh: 100 | flow_count: 13| func_call: True 
574.

110.jpg | 109.jpg | IOU: 90.21013070409253 |thresh: 99 | flow_count: 105| func_call: True 
111.jpg | 110.jpg | IOU: 82.10374252852269 |thresh: 99 | flow_count: 58| func_call: True 
112.jpg | 111.jpg | IOU: 69.41831775907376 |thresh: 99 | flow_count: 64| func_call: True 
113.jpg | 112.jpg | IOU: 84.85806464191363 |thresh: 99 | flow_count: 71| func_call: True 
114.jpg | 113.jpg | IOU: 91.5877173945069 |thresh: 99 | flow_count: 74| func_call: True 
115.jpg | 114.jpg | IOU: 82.95759756584202 |thresh: 99 | flow_count: 70| func_call: True 
116.jpg | 115.jpg | IOU: 88.65001517698236 |thresh: 99 | flow_count: 81| func_call: True 
117.jpg | 116.jpg | IOU: 80.9323364999588 |thresh: 99 | flow_count: 71| func_call: True 
118.jpg | 117.jpg | IOU: 69.86641943959219 |thresh: 99 | flow_count: 73| func_call: True 
119.jpg | 118.jpg | IOU: 86.09624561225752 |thresh: 99 | flow_count: 42| func_call: True 
120.jpg | 119.jpg | IOU: 75.6808761604738 |thresh: 99 | flow_count: 107| func_call: True 
121.jpg | 1

930.jpg | 929.jpg | IOU: 89.32539388267975 |thresh: 99 | flow_count: 60| func_call: True 
931.jpg | 930.jpg | IOU: 87.94522179473456 |thresh: 99 | flow_count: 49| func_call: True 
932.jpg | 931.jpg | IOU: 96.10399668707649 |thresh: 99 | flow_count: 48| func_call: True 
933.jpg | 932.jpg | IOU: 91.84422867016848 |thresh: 99 | flow_count: 42| func_call: True 
934.jpg | 933.jpg | IOU: 82.77118256103861 |thresh: 99 | flow_count: 50| func_call: True 
935.jpg | 934.jpg | IOU: 89.61208537113608 |thresh: 99 | flow_count: 16| func_call: True 
936.jpg | 935.jpg | IOU: 87.2180450710984 |thresh: 99 | flow_count: 10| func_call: True 
937.jpg | 936.jpg | IOU: 75.93772067256876 |thresh: 99 | flow_count: 20| func_call: True 
938.jpg | 937.jpg | IOU: 74.51024993758496 |thresh: 99 | flow_count: 22| func_call: True 
939.jpg | 938.jpg | IOU: 70.69155492877721 |thresh: 99 | flow_count: 27| func_call: True 
940.jpg | 939.jpg | IOU: 76.5716592702415 |thresh: 99 | flow_count: 32| func_call: True 
941.jpg | 94

310.jpg | 309.jpg | IOU: 85.48895432077413 |thresh: 98 | flow_count: 8| func_call: True 
311.jpg | 310.jpg | IOU: 74.43217277484813 |thresh: 98 | flow_count: 4| func_call: True 
312.jpg | 311.jpg | IOU: 72.81483416035519 |thresh: 98 | flow_count: 2| func_call: True 
313.jpg | 312.jpg | IOU: 95.64492053162756 |thresh: 98 | flow_count: 4| func_call: True 
314.jpg | 313.jpg | IOU: 97.03661808598463 |thresh: 98 | flow_count: 1| func_call: True 
315.jpg | 314.jpg | IOU: 95.55262708385152 |thresh: 98 | flow_count: 7| func_call: True 
316.jpg | 315.jpg | IOU: 91.715236896326 |thresh: 98 | flow_count: 8| func_call: True 
317.jpg | 316.jpg | IOU: 93.52214767346587 |thresh: 98 | flow_count: 7| func_call: True 
318.jpg | 317.jpg | IOU: 93.94747715414263 |thresh: 98 | flow_count: 6| func_call: True 
319.jpg | 318.jpg | IOU: 94.39452923239855 |thresh: 98 | flow_count: 1| func_call: True 
320.jpg | 319.jpg | IOU: 84.63192073538318 |thresh: 98 | flow_count: 2| func_call: True 
321.jpg | 320.jpg | IOU

...

445.jpg | 439.jpg | IOU: 82.56166149513777 |thresh: 75 | flow_count: 16| func_call: False 
446.jpg | 439.jpg | IOU: 81.16886038024369 |thresh: 75 | flow_count: 22| func_call: False 
447.jpg | 439.jpg | IOU: 81.00209297257851 |thresh: 75 | flow_count: 28| func_call: False 
448.jpg | 439.jpg | IOU: 65.11627906976744 |thresh: 75 | flow_count: 28| func_call: True 
449.jpg | 448.jpg | IOU: 50.799220272904485 |thresh: 75 | flow_count: 1| func_call: True 
450.jpg | 449.jpg | IOU: 56.67222201747956 |thresh: 75 | flow_count: 8| func_call: True 
451.jpg | 450.jpg | IOU: 96.80611955216116 |thresh: 75 | flow_count: 0| func_call: False 
452.jpg | 450.jpg | IOU: 97.67378570727863 |thresh: 75 | flow_count: 9| func_call: False 
453.jpg | 450.jpg | IOU: 87.235592528295 |thresh: 75 | flow_count: 8| func_call: False 
454.jpg | 450.jpg | IOU: 98.83720930232558 |thresh: 75 | flow_count: 17| func_call: False 
455.jpg | 450.jpg | IOU: 87.83556726620078 |thresh: 75 | flow_count: 19| func_call: False 
456.jpg 

[INFO] starting treshold value of 50
0.jpg | 0.jpg | IOU: 100.0 |thresh: 50 | flow_count: 0| func_call: False 
1.jpg | 0.jpg | IOU: 94.72719359200363 |thresh: 50 | flow_count: 0| func_call: False 
2.jpg | 0.jpg | IOU: 74.76198296116215 |thresh: 50 | flow_count: 0| func_call: False 
3.jpg | 0.jpg | IOU: 74.96126006944044 |thresh: 50 | flow_count: 0| func_call: False 
4.jpg | 0.jpg | IOU: 95.61440298402508 |thresh: 50 | flow_count: 3| func_call: False 
5.jpg | 0.jpg | IOU: 95.16921065902588 |thresh: 50 | flow_count: 5| func_call: False 
6.jpg | 0.jpg | IOU: 76.62547954449991 |thresh: 50 | flow_count: 13| func_call: False 
7.jpg | 0.jpg | IOU: 75.76963442107744 |thresh: 50 | flow_count: 15| func_call: False 
8.jpg | 0.jpg | IOU: 95.14883464666671 |thresh: 50 | flow_count: 15| func_call: False 
9.jpg | 0.jpg | IOU: 75.87938518726445 |thresh: 50 | flow_count: 13| func_call: False 
10.jpg | 0.jpg | IOU: 76.3593934293572 |thresh: 50 | flow_count: 13| func_call: False 
11.jpg | 0.jpg | IOU: 75

664.jpg | 662.jpg | IOU: 62.37603372268881 |thresh: 50 | flow_count: 39| func_call: False 
665.jpg | 662.jpg | IOU: 62.16694779419786 |thresh: 50 | flow_count: 41| func_call: False 
666.jpg | 662.jpg | IOU: 48.12378167641326 |thresh: 50 | flow_count: 52| func_call: True 
667.jpg | 666.jpg | IOU: 88.83271251361704 |thresh: 50 | flow_count: 14| func_call: False 
668.jpg | 666.jpg | IOU: 48.84768740031898 |thresh: 50 | flow_count: 15| func_call: True 
669.jpg | 668.jpg | IOU: 74.19762137504074 |thresh: 50 | flow_count: 9| func_call: False 
670.jpg | 668.jpg | IOU: 65.28016009148085 |thresh: 50 | flow_count: 9| func_call: False 
671.jpg | 668.jpg | IOU: 85.662177328844 |thresh: 50 | flow_count: 9| func_call: False 
672.jpg | 668.jpg | IOU: 82.75862068965517 |thresh: 50 | flow_count: 9| func_call: False 
673.jpg | 668.jpg | IOU: 80.99104008258738 |thresh: 50 | flow_count: 11| func_call: False 
674.jpg | 668.jpg | IOU: 62.48463942138895 |thresh: 50 | flow_count: 12| func_call: False 
675.jpg

327.jpg | 269.jpg | IOU: 26.070916340940613 |thresh: 25 | flow_count: 234| func_call: False 
328.jpg | 269.jpg | IOU: 25.818867398834126 |thresh: 25 | flow_count: 231| func_call: False 
329.jpg | 269.jpg | IOU: 25.194549387334664 |thresh: 25 | flow_count: 234| func_call: False 
330.jpg | 269.jpg | IOU: 25.52104479791804 |thresh: 25 | flow_count: 231| func_call: False 
331.jpg | 269.jpg | IOU: 25.178296279991194 |thresh: 25 | flow_count: 231| func_call: False 
332.jpg | 269.jpg | IOU: 26.59839790332467 |thresh: 25 | flow_count: 222| func_call: False 
333.jpg | 269.jpg | IOU: 26.59839790332467 |thresh: 25 | flow_count: 221| func_call: False 
334.jpg | 269.jpg | IOU: 26.835295045868925 |thresh: 25 | flow_count: 219| func_call: False 
335.jpg | 269.jpg | IOU: 26.546184879722677 |thresh: 25 | flow_count: 222| func_call: False 
336.jpg | 269.jpg | IOU: 26.638174033902686 |thresh: 25 | flow_count: 221| func_call: False 
337.jpg | 269.jpg | IOU: 26.852965139023983 |thresh: 25 | flow_count: 232

888.jpg | 870.jpg | IOU: 29.927138936163505 |thresh: 25 | flow_count: 176| func_call: False 
889.jpg | 870.jpg | IOU: 28.737832993040353 |thresh: 25 | flow_count: 167| func_call: False 
890.jpg | 870.jpg | IOU: 28.596437281537025 |thresh: 25 | flow_count: 164| func_call: False 
891.jpg | 870.jpg | IOU: 28.367842711584814 |thresh: 25 | flow_count: 158| func_call: False 
892.jpg | 870.jpg | IOU: 27.636261419872266 |thresh: 25 | flow_count: 157| func_call: False 
893.jpg | 870.jpg | IOU: 27.390713673435464 |thresh: 25 | flow_count: 162| func_call: False 
894.jpg | 870.jpg | IOU: 27.34053615107424 |thresh: 25 | flow_count: 161| func_call: False 
895.jpg | 870.jpg | IOU: 26.395217497407792 |thresh: 25 | flow_count: 161| func_call: False 
896.jpg | 870.jpg | IOU: 26.511192063781852 |thresh: 25 | flow_count: 158| func_call: False 
897.jpg | 870.jpg | IOU: 26.625258452845294 |thresh: 25 | flow_count: 160| func_call: False 
898.jpg | 870.jpg | IOU: 26.684450903739826 |thresh: 25 | flow_count: 1

In [15]:
with open('dynaimic_array_640x360.data', 'wb') as f:
    pickle.dump(dp, f)

In [16]:
df_640x360.to_csv('IOU_scores_detection_flag_flow_640x360.csv',index=False)

In [9]:
df_data=pd.read_csv('IOU_scores_detection_flag_flow_640x360.csv')

In [10]:
def divide_chunks(l, n): 
    for i in range(0, len(l), n):  
        yield l[i:i + n]
    
def compare_vectors_using_sliding_window(detection_vector_iou,detection_vector_flow_count):    
    
    #print(len(detection_vector_iou))
    #print(len(detection_vector_ssim))
    
    matches=0
    bottom_left=0
    top_right = 0
    bottom_right=0
    result_vector_iou=[]
    result_vector_flow_count=[]
    
    for i in range(0,len(detection_vector_iou)):
        if sum(detection_vector_iou[i])>0:
            result_vector_iou.append(1)
        else:                
            result_vector_iou.append(0)
            
        if sum(detection_vector_flow_count[i])>0:
            result_vector_flow_count.append(1)
        else:
            result_vector_flow_count.append(0)
        

        if sum(detection_vector_iou[i])>0 and sum(detection_vector_flow_count[i])>0:
            matches=matches+1
            
        elif sum(detection_vector_iou[i])==0 and sum(detection_vector_flow_count[i])>0:
            bottom_left=bottom_left+1
            
        if sum(detection_vector_iou[i])>0 and sum(detection_vector_flow_count[i])==0:
            top_right = top_right +1
        
        if sum(detection_vector_iou[i])==0 and sum(detection_vector_flow_count[i])==0:
            bottom_right = bottom_right+1
            
#    print(len(result_vector_iou),len(result_vector_ssim))
    #print(matches)
    
    return result_vector_iou,result_vector_flow_count,matches, bottom_left,top_right,bottom_right
    

In [11]:
per

In [12]:

iou_threshold=[100,]
percentiles_list=[50]

rows=[]
for i in iou_threshold[::-1]:
    print('Dealing with Threshold: ',i)
    temp_row=[]
    temp_row.append(i)
    matches,number_of_detections_per_percentile,flow_count_values_per_percentile=get_results(i,percentiles_list)
    print('matches',matches)
    print('no. of detections',number_of_detections_per_percentile)
    print('flow',flow_count_values_per_percentile)
    temp_row.append(number_of_detections_per_percentile)
    rows.append(temp_row)
    print(temp_row)

Dealing with Threshold:  100


NameError: name 'dp' is not defined

In [11]:
rows=[]
for i in iou_threshold[::-1]:
    #print('Dealing with Threshold: ',i)
    temp_row=[]
    temp_row.append(i)
    matches,number_of_detections_per_percentile,flow_count_values_per_percentile=get_results(i,percentiles_list)
    temp_row.append(matches)
    rows.append(temp_row)
    print(temp_row)

[100, [85, 78, 80, 70, 70, 65, 63, 63, 57, 50, 50, 52, 45, 48, 46, 46, 44, 35, 31, 17]]
[99, [85, 78, 80, 70, 70, 65, 60, 63, 57, 50, 50, 52, 44, 43, 46, 46, 44, 34, 31, 17]]
[98, [84, 79, 75, 71, 66, 64, 63, 57, 55, 52, 50, 56, 51, 48, 46, 46, 43, 36, 31, 17]]
[97, [84, 78, 75, 70, 65, 60, 63, 57, 52, 51, 52, 55, 48, 47, 46, 44, 42, 32, 31, 17]]
[96, [82, 77, 73, 71, 60, 63, 57, 52, 52, 50, 56, 44, 48, 46, 46, 44, 42, 33, 30, 17]]
[95, [80, 77, 71, 68, 57, 56, 50, 52, 50, 56, 45, 48, 48, 46, 46, 43, 42, 33, 31, 17]]
[94, [78, 77, 70, 68, 55, 50, 50, 50, 55, 55, 48, 43, 46, 46, 44, 43, 35, 33, 32, 17]]
[93, [78, 77, 70, 64, 53, 50, 50, 55, 55, 48, 43, 46, 46, 46, 44, 42, 34, 32, 29, 17]]
[92, [76, 76, 70, 63, 50, 50, 52, 55, 51, 48, 47, 46, 46, 44, 44, 43, 34, 32, 32, 16]]
[91, [77, 75, 65, 56, 52, 55, 55, 48, 43, 46, 46, 46, 46, 44, 41, 35, 32, 31, 28, 16]]
[90, [77, 75, 60, 57, 55, 53, 48, 48, 46, 46, 46, 46, 44, 44, 42, 35, 33, 32, 27, 16]]
[89, [77, 74, 60, 56, 55, 48, 48, 48, 47, 

In [88]:
options_v3_default=constants.options_default
net_default=cv2.dnn.readNetFromDarknet(options_v3_default['model'], options_v3_default['load'])
LABELS_default = open(options_v3_default['labelsPath']).read().strip().split("\n")
color = (255, 0, 0)

In [89]:
def predict_object(image_path,net,Labels,options):

    image = cv2.imread(image_path)
    (H, W) = image.shape[:2]

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(
        image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    

    # show timing information on YOLO
    

    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > options['confidence']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, options['confidence'], options['threshold'])
    detections=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            temp_list=[]
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,0), 3)
            #text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            #cv2.putText(image, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            

            temp_list.append(x)
            temp_list.append(y)
            temp_list.append(w)
            temp_list.append(h)
            temp_list.append(Labels[classIDs[i]])
            temp_list.append(confidences[i])
            temp_list.append(classIDs[i])
            
            
            
            detections.append(temp_list)
    
    return detections

In [90]:
def predict_object_image(image,net,Labels,options):


    (H, W) = image.shape[:2]

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    blob = cv2.dnn.blobFromImage(
        image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    layerOutputs = net.forward(ln)
    

    # show timing information on YOLO
    

    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > options['confidence']:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    idxs = cv2.dnn.NMSBoxes(boxes, confidences, options['confidence'], options['threshold'])
    detections=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            temp_list=[]
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            
            #cv2.rectangle(image, (x, y), (x + w, y + h), (0,255,0), 3)
            #text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
            #cv2.putText(image, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            

            temp_list.append(x)
            temp_list.append(y)
            temp_list.append(w)
            temp_list.append(h)
            temp_list.append(Labels[classIDs[i]])
            temp_list.append(confidences[i])
            temp_list.append(classIDs[i])
            
            
            
            detections.append(temp_list)
    
    return detections

In [94]:
start=time.time()
detections=predict_object('frames/2.jpg'.format(i),net_default,LABELS_default,options_v3_default)
print(time.time()-start)
print(detections)

0.5294549465179443
[[109, 217, 43, 133, 'person', 0.994403064250946, 0], [395, 291, 75, 228, 'person', 0.9706060886383057, 0], [233, 293, 75, 95, 'pottedplant', 0.9353656768798828, 58], [392, 335, 12, 9, 'cell phone', 0.7148078680038452, 67], [173, 242, 60, 78, 'pottedplant', 0.5509642958641052, 58]]


In [62]:
start = time.time()
count = 0
for i in range(0,1083):
    detections=predict_object('2560x1440/{}.jpg'.format(i),net_default,LABELS_default,options_v3_default)
    count = count + len(detections)
print(time.time()-start)
print(count)

KeyboardInterrupt: 

In [5]:
start = time.time()
count = 0
for i in range(0,1083):
    detections=predict_object('1280x720/{}.jpg'.format(i),net_default,LABELS_default,options_v3_default)
    count = count + len(detections)
print(time.time()-start)
print(count)

646.9290161132812
6298


In [8]:
start = time.time()
count = 0
for i in range(0,1083):
    detections=predict_object('640x360/{}.jpg'.format(i),net_default,LABELS_default,options_v3_default)
    
    count = count + len(detections)
print(time.time()-start)
print(count)

587.8774824142456
5781


In [98]:
start = time.time()
count =0 
for i in range(0,1083):
    predict_object('frames/{}.jpg'.format(i),net_default,LABELS_default,options_v3_default)
print(time.time()-start)

624.259843826294


In [36]:

def get_flow_vector_count(image_1,image_2):
    
    frame1 = cv2.imread(image_1)
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

    frame2 = cv2.imread(image_2)
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    count = 0
    for i in range(1, 36):
        for j in range(1,64):
            X1 = (i)*10
            Y1 = (j)*10
            
            X2 = int(X1 + flow[X1,Y1,1])
            Y2 = int(Y1 + flow[X1,Y1,0])
            
            
            if abs(X2-X1)>1 or abs(Y2-Y1)>1 and count:
                count+=1
            if count>638:
                return True
        
    return False

In [ ]:
start = time.time()
fixed = 0 
for i in range(0,1083):
    value = get_flow_vector_count('frames/{}.jpg'.format(i),'frames/{}.jpg'.format(fixed))
    if value:
        predict_object('frames/{}.jpg'.format(i),net_default,LABELS_default,options_v3_default)
        fixed=i
print(time.time()-start)

In [96]:

frame1=cv2.imread('frames/0.jpg')

def get_flow_vector_count_timer(new_image):
    
    global frame1
    prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

    next = cv2.cvtColor(new_image,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    count = 0
    for i in range(1, 54):
        for j in range(1,96):
            X1=i*10
            Y1=j*10
            X2 = int(X1 + flow[X1,Y1,1])
            Y2 = int(Y1+ flow[X1,Y1,0])
            
            
            if abs(X2-X1)>1 or abs(Y2-Y1)>1:
                count+=1
            if count>109:
                return True
            
    return False

                

    

In [97]:
start = time.time()

for i in range(0,1083):
    new_image=cv2.imread('frames/{}.jpg'.format(i))
   
    count = get_flow_vector_count_timer(new_image)
    if count:
        frame1 = new_image
        predict_object_image(new_image,net_default,LABELS_default,options_v3_default)
        
end = time.time()
print(end-start)


546.6609678268433
